# Задание Lite к занятию Методы Faiss, Langchain

Возьмите базу знаний "Правила безопасности опасных производственных объектов"

URL: https://docs.google.com/document/d/1q4l912Re8zuIfBax4FDS3ZppYmVPzER3Si2wrmznddc

или любой другой текстовый документ по ссылке с ГуглДиска.

- Разбейте текст на чанки с использованием CharacterTextSplitter
- Создайте векторную базу и сохраниете ее на ГуглДиск
- Загрузите из ГуглДиска сохраненную на предыдущем шаге векторную базу
- Создайте простого консультанта по базе знаний. Используйте стандартную библиотеку openai.
- В этом задании не используйте ретриверы и ChatOpenAI из Langchain.
- Задайте вопрос консультанту по базе знаний и получите ответ

In [1]:
# Устанавливаем необходимые библиотеки
!pip install -q openai==1.57.0 langchain==0.3.9 langchain_core==0.3.21 langchain_community==0.3.9 \
langchain-openai==0.2.11 faiss-cpu==1.9.0.post1 tiktoken==0.8.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00


In [4]:
# Импортируем необходимые библиотеки
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
import os
import re
import requests
import textwrap
import openai
import tiktoken
import matplotlib.pyplot as plt
from openai import OpenAI
from google.colab import userdata, drive

In [5]:
# Использование ключа API от ProxyAPI
key = userdata.get('PROXY_API_KEY')
os.environ["OPENAI_API_KEY"] = key

# Адрес сервера ProxyAPI
base_url = 'https://api.proxyapi.ru/openai/v1'
os.environ["OPENAI_BASE_URL"] = base_url

# Создание клиента OpenAI с использованием API ключа из переменных среды
client = OpenAI()

In [6]:
# Функции

# функция для загрузки документа Docx по ссылке из гугл драйв
def download_google_doc(url: str) -> str:
    # Извлекаем ID документа из URL
    match_ = re.search(r'/document/d/([a-zA-Z0-9-_]+)', url)
    if not match_:
        raise ValueError("Invalid Google Docs URL")
    doc_id = match_.group(1)
    download_url = f'https://docs.google.com/document/d/{doc_id}/export?format=txt'
    response = requests.get(download_url, stream=True)
    if response.status_code != 200:
        raise RuntimeError("Failed to download the document")
    return response.text


# Функция для форматирования текста по абзацам
def format_text(text, width=120):
    # Разделяем текст на абзацы
    paragraphs = text.split('\n')
    # Форматируем каждый абзац отдельно
    formatted_paragraphs = []
    for paragraph in paragraphs:
        # Используем textwrap.fill для форматирования абзаца, чтобы длина строки не превышала width
        formatted_paragraph = textwrap.fill(paragraph, width)
        formatted_paragraphs.append(formatted_paragraph)
    # Объединяем абзацы с символом новой строки
    return '\n'.join(formatted_paragraphs)


# Функция возвращает количество токенов в строке в зависимости от используемой модели
def num_tokens_from_string(string: str, model='gpt-4o-mini') -> int:
    # Получаем имя кодировки для указанной модели
    encoding_name = tiktoken.encoding_for_model(model).name
    # Получаем объект кодировки на основе имени кодировки
    encoding = tiktoken.get_encoding(encoding_name)
    # Кодируем строку и вычисляем количество токенов
    num_tokens = len(encoding.encode(string))
    # Возвращаем количество токенов
    return num_tokens


# Построение гистограммы распределения количества токенов по чанкам
def create_histogram(chunks):
    print("\nОбщее количество чанков: ", len(chunks))
    # Подсчет токенов для каждого чанка
    try: # для формата чанков LangChain Document
        chunk_token_counts = [num_tokens_from_string(chunk.page_content) for chunk in chunks]
    except: # для текстового формата чанков
        chunk_token_counts = [num_tokens_from_string(chunk) for chunk in chunks]
    # Строим гистограмму
    plt.figure(figsize=(7, 4)) # размер
    plt.hist(chunk_token_counts, bins=10, alpha=0.5, label='Чанки')
    plt.title('Распределение к-ва токенов по чанкам')  # Заголовок графика
    plt.xlabel('К-во токенов в чанке')  # Подпись оси X
    plt.ylabel('К-во чанков')  # Подпись оси Y
    plt.show()  # Отображаем график


# (CharacterTextSplitter) Формируем чанки из текста по количеству символов
def split_text(text: str,
               chunk_size=3000,    # Ограничение к-ва символов в чанке
               chunk_overlap=200): # к-во символов перекрытия в чанке
    # Удалить пустые строки и лишние пробелы
    text = re.sub(r'\s+', ' ', text).strip()
    # Создаем экземпляр CharacterTextSplitter с заданными парамаетрами
    splitter = CharacterTextSplitter(chunk_size=chunk_size,
                                     chunk_overlap=chunk_overlap,
                                     separator=" ")
    return splitter.split_text(text) # список текстовых чанков


# Создание индексной (векторной) базы из текстовых чанков и сохранение на диск
def create_db_index_from_text_save(chunks_text,
                                   index_name, # имя для индексной базы
                                   path):  # путь к папке
    # Создаем индексную базу с использованием FAISS ---------------------------------
    # FAISS.from_texts - из списка текстовых чанков
    db_index = FAISS.from_texts(chunks_text, OpenAIEmbeddings())
    # сохраняем индексную базу ------------------------------------------------------
    db_index.save_local(folder_path=path, # путь к папке (path)
                        index_name=index_name)  # имя для индексной базы (index_name)
    # --------------------------------------------------------------------------------
    return db_index


# Загрузка векторной базы с диска
def load_db_vector(folder_path_db_index,  # путь к сохраненной векторной базе
                   index_name):           # имя сохраненной векторной базы
    return FAISS.load_local(
                allow_dangerous_deserialization=True,
                embeddings=OpenAIEmbeddings(),
                folder_path=folder_path_db_index,
                index_name=index_name)


# Функция запроса и ответа от OpenAI с поиском по векторной базе данных
def generate_answer(query,    # запрос пользователя
                    db_index, # векторная база знаний
                    k=5,      # используемое к-во чанков
                    verbose=True, # выводить ли на экран выбранные чанки
                    model='gpt-4o-mini',
                    temp=0.1):
    # Поиск чанков по векторной базе данных
    similar_documents = db_index.similarity_search(query, k=k)
    # Формирование текстового контента из выбранных чанков для модели
    message_content = re.sub(r'\n{2}', ' ', '\n '.join(
        [f'Отрывок документа № {i+1}:\n' + doc.page_content
        for i, doc in enumerate(similar_documents)]))
    if verbose:
        print(message_content) # печать на экран выбранных чанков

    messages = [{"role": "system",
                 "content": f'Ответь на вопрос пользователя на основании информации из базы знаний: \n{message_content}'},
                {"role": "user",
                 "content": f'Вопрос пользователя: {query}'}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temp)
    return response.choices[0].message.content

In [7]:
data_from_url = download_google_doc('https://docs.google.com/document/d/1q4l912Re8zuIfBax4FDS3ZppYmVPzER3Si2wrmznddc')

In [8]:
formatted_data = format_text(data_from_url)
source_chank = split_text(formatted_data)

db_index = create_db_index_from_text_save(source_chank, 'db_index', '/content')

In [9]:
load_db_index = load_db_vector('/content', 'db_index')

In [10]:
query = 'Что является целью настоящих ФНП?'
ans = generate_answer(query, load_db_index)
ans

Отрывок документа № 1:
угрозы аварийной ситуации; г) при выявлении нарушений требований к эксплуатации ПС, изложенных в настоящих ФНП, принимать меры по их устранению и предупреждению, в том числе проводить внеочередную проверку знаний работников, допустивших такие нарушения. 25. Работники ОПО, непосредственно занимающиеся эксплуатацией ПС, должны соответствовать следующим требованиям: а) иметь выданное в порядке, установленном эксплуатирующей организацией, удостоверение на право самостоятельной работы по соответствующим видам деятельности; б) знать критерии работоспособности применяемых ПС в соответствии с требованиями руководства (инструкции) по эксплуатации применяемых ПС, используемых съемных грузозахватных приспособлений и тары, а также технологический процесс транспортировки грузов; в) в случае возникновения угрозы аварийной ситуации информировать об этом своего непосредственного руководителя; г) знать порядок действий по инструкциям эксплуатирующей организации в случае возникнов

'Целью настоящих Федеральных норм и правил в области промышленной безопасности (ФНП) является создание организационной основы обеспечения промышленной безопасности на опасных производственных объектах (ОПО), на которых используются подъемные сооружения (ПС), направленной на предотвращение и/или минимизацию последствий аварий и инцидентов, с учетом индивидуального риска потери жизни и здоровья людей, участвующих в процессах монтажа, наладки, эксплуатации, обслуживания, ремонта, реконструкции, модернизации и утилизации (ликвидации) ПС.'